# Analysis for Insight Project

## Setup

In [1]:
# import modules
import pandas as pd
import numpy as np
import rootpath
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import qgrid

# Set root path for project
path = rootpath.detect()

# Read in data
# Use list comprehension to read in all files
df = pd.read_csv(f"{path}/data/data_clean.csv", index_col=0)

In [2]:
df.head()

,product,Unnamed: 2,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,*95% naturally derived/dérivé naturellement.,*avena sativa (oat) kernel extract,...,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root,dry,normal,oily,total_ingred
0,1000 Roses Cleansing Foam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,29.0
1,3% Glycolic Solutions Cleanser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,48.0
2,82% Organic Softening Cleanser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,42.0
3,AHA/BHA Acne Clearing Gel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,33.0
4,AHA/BHA Exfoliating Cleanser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,42.0


In [5]:
features = df.copy().set_index('product')

,Unnamed: 2,(thyme) leaf extract,* aloe barbadensis (aloe vera) leaf juice,* cymbopogon schoenanthus (lemongrass) oil,* potassium cocotte (saponified coconut oil*),* rosmarinus officials (rosemary) leaf oil,* tocopherol (vitamin e). *certified usda organic,*95% naturally derived/dérivé naturellement.,*avena sativa (oat) kernel extract,*baptisia tinctoria root extract,...,zinc sulfate.,zingiber officinale (ginger) oil,zingiber officinale (ginger) root,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root oil*,zingiber officinale root,dry,normal,oily,total_ingred
product,,,,,,,,,,,,,,,,,,,,,
1000 Roses Cleansing Foam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,29.0
3% Glycolic Solutions Cleanser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,48.0
82% Organic Softening Cleanser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,42.0
AHA/BHA Acne Clearing Gel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,33.0
AHA/BHA Exfoliating Cleanser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,42.0


## K-means++

In [ ]:
# Set cluster arguments
kmeans_kwargs = {
        "init": "k-means++",
        "n_init": 10,
        "max_iter": 300,
        "random_state": 42,
        }

In [ ]:
silhouette_coefficients = []

# Notice you start at 2 clusters for silhouette coefficient
for cluster in range(2, 50):
    kmeans = KMeans(n_clusters=cluster, **kmeans_kwargs)
    kmeans.fit(features)
    score = silhouette_score(scaled_features, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()